<a href="https://colab.research.google.com/github/Han529/ML-Data-Science-Apps/blob/master/Junior_Data_Analytics_Assessment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ==============================================================================
#                            Imports and Setup
# ==============================================================================
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
import random
import re
import numpy as np
import os
from datetime import datetime
from urllib.parse import urljoin
import json
import logging
from typing import List, Dict, Optional, Tuple, Any # For type hints

# --- Configuration ---

# Logging Setup
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)-8s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)
# Suppress overly verbose logs from underlying libraries like urllib3
logging.getLogger("urllib3").setLevel(logging.WARNING)
logging.getLogger("requests").setLevel(logging.WARNING)

# --- Constants ---
BASE_URL: str = "https://13f.info"
DEFAULT_USER_AGENT: str = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
CSV_FILENAME: str = "Fund Manager Shares Analysis.csv"

# Define the exact columns desired in the final output, in order
TARGET_FINAL_COLUMNS: List[str] = [
    'fund_name',
    'filing_date',
    'quarter',
    'stock symbol', # Note: Renamed from 'sym' or 'stock_symbol'
    'cl',
    'value_usd_000', # Note: Assumes header cleaning produces this
    'shares',
    'change',
    'pct_change',
    'inferred_transaction_type'
]

logging.info("Script started. Libraries imported and logging configured.")

# ==============================================================================
#                        Scraping Function Definitions
# ==============================================================================

# ***** CORRECTED scrape_manager_links FUNCTION *****
def scrape_manager_links(url: str) -> List[str]:
    """
    Scrapes relative links for individual managers from the main 13f.info page.

    Args:
        url (str): The URL of the page listing managers (e.g., BASE_URL + "/").

    Returns:
        List[str]: A list of unique relative URL paths (e.g., '/manager/...')
                   or an empty list if scraping fails.
    """
    logging.info(f"Scraping manager links from: {url}")
    headers = {'User-Agent': DEFAULT_USER_AGENT}
    try:
        # Fetch the page content
        response = requests.get(url, headers=headers, timeout=30)
        response.raise_for_status() # Check for HTTP errors (4xx, 5xx)

        # Parse the HTML
        soup = BeautifulSoup(response.text, 'html.parser')
        links = soup.find_all('a', href=True)

        # Filter for relative links starting with '/manager/'
        # Ensure href exists and is a string before checking startswith
        # Use a set for automatic uniqueness, then convert back to list
        manager_links_relative = list(set(
            link['href'] for link in links
            if isinstance(link.get('href'), str) and link['href'].startswith('/manager/')
        ))

        logging.info(f"Found {len(manager_links_relative)} unique relative manager links.")
        return manager_links_relative

    except requests.exceptions.RequestException as e:
        # Handle network/HTTP errors during the request
        logging.error(f"Network or HTTP error scraping manager links from {url}. Error: {e}")
        return [] # Return empty list on failure
    except Exception as e:
        # Handle any other unexpected errors (e.g., during parsing)
        logging.error(f"Unexpected error scraping manager links from page {url}. Error: {e}")
        return [] # Return empty list on failure
# ***** END OF CORRECTED scrape_manager_links FUNCTION *****

# ---

def scrape_quarter_links_from_manager(relative_manager_link: str, base_url: str) -> List[str]:
    """
    Scrapes the absolute URLs for quarterly filings from a specific manager's page.
    Looks for links within the 'Quarter' column of the main table.

    Args:
        relative_manager_link (str): The relative path of the manager's page.
        base_url (str): The base URL of the site (e.g., "https://13f.info").

    Returns:
        List[str]: A list of absolute URLs for quarterly filings, or empty list on failure.
    """
    full_url = urljoin(base_url, relative_manager_link)
    logging.info(f"Scraping quarter links from: {full_url}")
    headers = {'User-Agent': DEFAULT_USER_AGENT}
    quarter_links = []
    try:
        response = requests.get(full_url, headers=headers, timeout=30)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        # Find the primary data table (more flexible selector)
        table = soup.find('table', {'class': re.compile(r'\btable\b')}) # Find any table with 'table' class
        if not table:
            logging.warning(f"No data table found on manager page {full_url}.")
            return []

        # Find 'Quarter' column index robustly
        header_row = table.find('thead') or table.find('tr')
        if not header_row:
            logging.warning(f"No table header found on {full_url}.")
            return []
        headers_list = header_row.find_all(['th', 'td'])
        quarter_col_index = -1
        for i, header in enumerate(headers_list):
            # Use lower() for case-insensitive comparison
            if header.get_text(strip=True).lower() == "quarter":
                quarter_col_index = i
                break
        if quarter_col_index == -1:
            logging.warning(f"'Quarter' column header not found in table on {full_url}.")
            return []

        # Find data rows (tbody or trs after header)
        tbody = table.find('tbody')
        data_rows = tbody.find_all('tr') if tbody else (table.find_all('tr')[1:] if len(table.find_all('tr')) > 1 else [])
        if not data_rows:
            logging.warning(f"No data rows found in table on {full_url}.")
            return []

        # Extract links from the correct column
        for row in data_rows:
            cells = row.find_all('td')
            if len(cells) > quarter_col_index:
                link_tag = cells[quarter_col_index].find('a', href=True)
                # Check if href exists and is a non-empty string
                href_val = link_tag.get('href') if link_tag else None
                if isinstance(href_val, str) and href_val.strip():
                    quarter_links.append(urljoin(base_url, href_val)) # Ensure absolute URL

        logging.info(f"Found {len(quarter_links)} quarter links on {full_url}.")
        return quarter_links
    except requests.exceptions.RequestException as e:
        logging.error(f"Network/HTTP error retrieving manager page {full_url} for quarter links. Error: {e}")
        return []
    except Exception as e:
        logging.error(f"Unexpected error parsing manager page {full_url} for quarter links. Error: {e}")
        return []

# ---

def scrape_fund_name(manager_page_url: str) -> str:
    """
    Scrapes the fund name from the manager's main page (typically H1 tag).
    Includes fallback logic to parse name from URL path if H1 is missing.

    Args:
        manager_page_url (str): The absolute URL of the manager's overview page.

    Returns:
        str: The extracted fund name, or "Unknown Fund Name" if extraction fails.
    """
    logging.info(f"Scraping fund name from: {manager_page_url}")
    headers = {'User-Agent': DEFAULT_USER_AGENT}
    default_name = "Unknown Fund Name"
    try:
        response = requests.get(manager_page_url, headers=headers, timeout=25)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'lxml') # Use lxml for parsing efficiency
        h1_tag = soup.find('h1')
        if h1_tag:
            name = h1_tag.get_text(strip=True)
            # Basic cleaning to remove common suffixes like CIK number
            if ' CIK#' in name: name = name.split(' CIK#')[0].strip()
            logging.info(f"Found fund name via H1: {name}")
            return name if name else default_name
        else: # Fallback to URL parsing
            logging.warning(f"H1 tag not found on {manager_page_url}. Attempting URL parsing for name.")
            try:
                path_parts = manager_page_url.strip('/').split('/')
                # Expecting format like ['https:', '', '13f.info', 'manager', '000...-fund-name-inc']
                if len(path_parts) > 3 and path_parts[-2] == 'manager':
                    url_name_part = path_parts[-1]
                    # Find first dash to separate ID from name part
                    first_dash_index = url_name_part.find('-')
                    if first_dash_index != -1:
                         # Take part after first dash, remove leading/trailing dashes, title case
                         parsed_name = url_name_part[first_dash_index:].strip('-').replace('-', ' ').title()
                         if parsed_name:
                            logging.info(f"Using parsed name from URL: {parsed_name}")
                            return parsed_name
            except Exception as url_parse_e: # Catch potential errors during list indexing/splitting
                 logging.warning(f"URL parsing for name failed: {url_parse_e}")
            logging.warning(f"Could not determine fund name for {manager_page_url}.")
            return default_name
    except requests.exceptions.RequestException as e:
        logging.error(f"Network/HTTP error retrieving {manager_page_url} for name. Error: {e}")
        return default_name
    except Exception as e:
        logging.error(f"Unexpected error parsing {manager_page_url} for name: {e}")
        return default_name

# ---

def _clean_header(header_text: str) -> str:
    """Helper function to consistently clean table header text."""
    if not isinstance(header_text, str): return ""
    text = header_text.lower().strip()
    text = text.replace(' ', '_')
    text = text.replace('($000)', 'usd_000') # Specific replacement for value column
    text = text.replace('%', 'pct')
    # Remove any remaining non-alphanumeric characters except underscore
    text = re.sub(r'[^\w_]+', '', text)
    return text

# ---

def _extract_headers(table_tag: BeautifulSoup) -> List[str]:
    """Robustly extracts and cleans header text from a table tag."""
    header_cells = []
    # Strategy 1: thead > tr > th/td
    thead = table_tag.find('thead')
    if thead:
        header_row = thead.find('tr')
        if header_row: header_cells = header_row.find_all(['th', 'td'])
        # Strategy 2: thead > th (less common)
        if not header_cells: header_cells = thead.find_all('th')
    # Strategy 3: Fallback to first tr in table > th/td
    if not header_cells:
        first_row = table_tag.find('tr')
        if first_row: header_cells = first_row.find_all(['th', 'td'])

    if not header_cells:
        logging.warning("Could not find any header cells (th/td) using multiple strategies.")
        return []

    headers = [_clean_header(h.get_text()) for h in header_cells]
    # Filter out empty headers that might result from cleaning
    headers = [h for h in headers if h]
    logging.info(f"Extracted {len(headers)} headers: {headers}")
    return headers

# --- Helper function for direct BS4 parsing (used as fallback) ---
def parse_tables_directly_bs4(soup_or_table_tag: Any, url: str, single_table: bool = False) -> List[pd.DataFrame]:
    """
    Parses table(s) directly from HTML using BeautifulSoup. Used as a fallback.
    Includes robust header finding and column mismatch handling.

    Args:
        soup_or_table_tag: BeautifulSoup soup object or a specific table Tag.
        url (str): The source URL (for logging and SourceURL column).
        single_table (bool): If True, treats input as a single table Tag.

    Returns:
        List[pd.DataFrame]: A list of DataFrames created from the parsed tables.
    """
    dataframes = []
    potential_tables = []
    if single_table:
        potential_tables = [soup_or_table_tag] if soup_or_table_tag else []
        mode = "single table"
    else:
        # More flexible selector for finding tables
        potential_tables = soup_or_table_tag.find_all('table', {'class': re.compile(r'\btable\b')}) if soup_or_table_tag else []
        mode = "all tables"
    logging.info(f"[Direct BS4 - {mode}] Parsing {len(potential_tables)} potential table(s) on {url}")

    if not potential_tables:
        logging.warning(f"[Direct BS4 - {mode}] No tables found to parse on {url}")
        return []

    for i, table_tag in enumerate(potential_tables):
        logging.debug(f"[Direct BS4] Processing table {i+1}")
        headers_list = _extract_headers(table_tag) # Use robust header extraction
        if not headers_list:
            logging.warning(f"[Direct BS4] Skipping table {i+1} on {url} - No valid headers extracted.")
            continue

        tbody = table_tag.find('tbody')
        rows_in_body = tbody.find_all('tr') if tbody else (table_tag.find_all('tr')[1:] if len(table_tag.find_all('tr')) > 1 else [])
        if not rows_in_body:
             logging.warning(f"[Direct BS4] Skipping table {i+1} on {url} - Headers found but no data rows (tbody/tr).")
             continue

        data_rows = []
        expected_cols = len(headers_list)
        column_mismatch_logged = False
        for row_idx, row in enumerate(rows_in_body):
            cells = row.find_all('td')
            actual_cols = len(cells)
            if actual_cols == expected_cols:
                # Extract text, handle potential None values gracefully
                row_data = [c.get_text(strip=True) if c else None for c in cells]
                data_rows.append(row_data)
            else:
                 # Log only once per table for mismatches
                 if not column_mismatch_logged:
                    logging.warning(f"[Direct BS4] Table {i+1}: Row {row_idx} has {actual_cols} cells, expected {expected_cols}. Skipping mismatched rows for this table on {url}.")
                    column_mismatch_logged = True

        if data_rows:
            try:
                df = pd.DataFrame(data_rows, columns=headers_list)
                df['SourceURL'] = url
                # Replace purely whitespace cells with NaN for better processing later
                df = df.replace(r'^\s*$', np.nan, regex=True)
                dataframes.append(df)
                logging.info(f"[Direct BS4] Successfully created DataFrame (Shape: {df.shape}) from table {i+1} on {url}")
            except Exception as e:
                 # Catch potential errors during DataFrame creation (e.g., duplicate columns)
                 logging.error(f"[Direct BS4] Error creating DataFrame for table {i+1} on {url}: {e}", exc_info=True)
        else:
            logging.warning(f"[Direct BS4] Extracted headers but no valid data rows constructed for table {i+1} on {url}")
    return dataframes

# --- Main Data Scraping Function ---
def scrape_table_data_and_metadata(url: str) -> Dict[str, Any]:
    """
    Scrapes table data, filing date, and quarter title from a given URL.
    Prioritizes AJAX data loading if table has 'data-url' attribute.
    Falls back to direct HTML parsing if AJAX fails or is not applicable.

    Args:
        url (str): The absolute URL of the quarterly filing page.

    Returns:
        Dict[str, Any]: A dictionary containing:
            'dataframes': List[pd.DataFrame] - List of extracted data tables.
            'filing_date': Optional[str] - The extracted filing date.
            'quarter': Optional[str] - The extracted quarter title.
    """
    logging.info(f"Scraping table data & metadata from: {url}")
    # Prepare headers for both initial HTML and potential AJAX request
    headers = {'User-Agent': DEFAULT_USER_AGENT, 'Accept': 'application/json, text/javascript, */*; q=0.01', 'X-Requested-With': 'XMLHttpRequest'}
    result: Dict[str, Any] = {'dataframes': [], 'filing_date': None, 'quarter': None}
    html_content = None

    # 1. Fetch Initial HTML Page
    try:
        response_page = requests.get(url, headers=headers, timeout=40) # Increased timeout
        response_page.raise_for_status()
        html_content = response_page.text
    except requests.exceptions.RequestException as e:
        logging.error(f"Failed to retrieve page {url}. Status: {getattr(e.response, 'status_code', 'N/A')}. Error: {e}")
        return result # Cannot proceed without HTML
    except Exception as e:
        logging.error(f"Unexpected error during page fetch for {url}: {e}")
        return result

    # 2. Parse HTML & Extract Metadata (if HTML fetched)
    soup = None
    try:
        soup = BeautifulSoup(html_content, 'lxml')
        # Filing Date
        date_dt = soup.find('dt', string=lambda t: t and 'Date filed' in t.strip())
        if date_dt and date_dt.find_next_sibling('dd'):
            result['filing_date'] = date_dt.find_next_sibling('dd').get_text(strip=True)
        # Quarter Title (broader search)
        qtr_h = soup.find(['h1', 'h2', 'h3'], string=lambda t: t and ('13F Holdings' in t or 'Quarter' in t))
        if qtr_h: result['quarter'] = qtr_h.get_text(strip=True)
        logging.info(f"Metadata extracted - Date: {result['filing_date']}, Quarter: {result['quarter']}")
    except Exception as e:
        logging.error(f"Error parsing HTML or extracting metadata from {url}: {e}")
        # Continue, attempt table parsing anyway

    # 3. Find Target Table (usually the main data table)
    target_table = soup.find('table', id='filingAggregated') if soup else None

    if not target_table:
        logging.warning(f"Target table (id='filingAggregated') not found. Parsing all tables on page: {url}")
        if soup: result['dataframes'] = parse_tables_directly_bs4(soup, url) # Parse all tables as fallback
        return result

    # 4. Extract HTML Headers (Needed for both AJAX and direct parsing)
    headers_list = _extract_headers(target_table)
    if not headers_list:
        logging.error(f"CRITICAL: Failed to extract HTML headers from target table on {url}. Cannot reliably parse data.")
        # Try direct parsing anyway, it might find headers differently, but less reliable
        result['dataframes'] = parse_tables_directly_bs4(target_table, url, single_table=True)
        return result

    # 5. Check for AJAX 'data-url'
    data_url_path = target_table.get('data-url')
    ajax_succeeded = False

    if data_url_path:
        ajax_url = urljoin(BASE_URL, data_url_path)
        logging.info(f"AJAX endpoint detected. Attempting fetch from: {ajax_url}")
        try:
            response_ajax = requests.get(ajax_url, headers=headers, timeout=40)
            response_ajax.raise_for_status() # Check for server errors (like 500)
            ajax_data = response_ajax.json()
            table_rows_data = ajax_data.get('data') # Standard key for DataTables

            if table_rows_data and isinstance(table_rows_data, list):
                expected_cols = len(headers_list)
                cleaned_rows = []
                mismatched_logged = False
                for i, row in enumerate(table_rows_data):
                    if isinstance(row, list):
                        actual_cols = len(row)
                        if actual_cols >= expected_cols:
                            processed_row = row[:expected_cols] # Slice if too many cols
                            if actual_cols > expected_cols and not mismatched_logged:
                                logging.warning(f"AJAX row {i} column count ({actual_cols}) > header count ({expected_cols}). Using first {expected_cols}.")
                                mismatched_logged = True
                        else: # actual_cols < expected_cols
                            if not mismatched_logged:
                                logging.warning(f"AJAX row {i} column count ({actual_cols}) < header count ({expected_cols}). Skipping row.")
                                mismatched_logged = True
                            continue # Skip row

                        # Clean HTML within cells
                        cleaned_cells = [BeautifulSoup(str(c), 'lxml').get_text(strip=True) if isinstance(c, str) else c for c in processed_row]
                        cleaned_rows.append(cleaned_cells)
                    else:
                        logging.warning(f"Skipping AJAX row {i} due to non-list format: {type(row)}")

                if cleaned_rows:
                    # Create DataFrame using HTML headers and cleaned AJAX data
                    df = pd.DataFrame(cleaned_rows, columns=headers_list)
                    df['SourceURL'] = url
                    result['dataframes'].append(df)
                    ajax_succeeded = True
                    logging.info(f"Successfully created DataFrame (Shape: {df.shape}) from AJAX data.")
                else:
                    logging.warning(f"Processed AJAX response but no valid rows constructed for {url}.")
            else:
                logging.warning(f"AJAX response missing 'data' key or 'data' is not a list for {ajax_url}.")

        except requests.exceptions.RequestException as e:
            # Log specific HTTP error if available
            http_status = getattr(e.response, 'status_code', 'N/A')
            logging.error(f"AJAX request failed for {ajax_url}. Status: {http_status}. Error: {e}") # Log 500 error here
        except json.JSONDecodeError as e:
            logging.error(f"Failed to decode JSON from {ajax_url}. Error: {e}")
        except Exception as e:
            logging.error(f"Unexpected error processing AJAX for {url}. Error: {e}", exc_info=True)

        # If AJAX path was taken and succeeded, return result now
        if ajax_succeeded:
            return result
        else:
            logging.warning(f"AJAX processing did not yield a DataFrame for {url}.") # Fall through to direct parse

    # 6. Fallback to Direct Parsing (if no data-url OR if AJAX failed)
    logging.info(f"Falling back to direct HTML parsing for target table on: {url}")
    result['dataframes'] = parse_tables_directly_bs4(target_table, url, single_table=True)
    return result


# ==============================================================================
#                        Execution Stage 1: Get Manager Links
# ==============================================================================
print("\n" + "="*70)
print(" Stage 1: Scraping Manager Links")
print("="*70)

manager_links_relative: List[str] = scrape_manager_links(BASE_URL + "/")
print(f"Found {len(manager_links_relative)} unique relative manager links.")

if not manager_links_relative:
    print("No manager links found. Exiting script.")
    exit()

# Optional: Limit managers for testing
# test_limit = 10
# print(f"\n*** LIMITING MANAGERS TO {test_limit} FOR TESTING ***\n")
# manager_links_relative = manager_links_relative[:test_limit]

time.sleep(random.uniform(0.5, 1.0)) # Small delay before next stage


# ==============================================================================
#                 Execution Stage 2: Get Quarter Links per Manager
# ==============================================================================
print("\n" + "="*70)
print(" Stage 2: Scraping Quarter Links for Each Manager")
print("="*70)

manager_quarter_links: Dict[str, List[str]] = {}
num_managers = len(manager_links_relative)

for i, rel_link in enumerate(manager_links_relative):
    print(f"\n[{i+1}/{num_managers}] Getting quarter links for Manager: {rel_link}")
    links = scrape_quarter_links_from_manager(rel_link, BASE_URL)
    manager_quarter_links[rel_link] = links # Store links (or empty list if failed)
    if not links:
        logging.warning(f"No quarter links found or error occurred for {rel_link}")
    # Polite delay between manager page requests
    time.sleep(random.uniform(1.0, 3.0))

print(f"\nFinished getting quarter links for {len(manager_quarter_links)} managers processed.")


# ==============================================================================
#                 Execution Stage 3: Scrape Table Data & Metadata
# ==============================================================================
print("\n" + "="*70)
print(" Stage 3: Scraping Table Data and Metadata for Each Filing")
print("="*70)

all_scraped_dataframes: List[pd.DataFrame] = [] # List to hold all successfully scraped DataFrames
total_managers_to_scrape = len(manager_quarter_links)

for i, (manager_link, quarter_links) in enumerate(manager_quarter_links.items()):
    print(f"\n>>> Processing Manager {i+1}/{total_managers_to_scrape}: {manager_link} <<<")
    logging.info(f"Processing Manager [{i+1}/{total_managers_to_scrape}]: {manager_link}")

    # Get Fund Name (once per manager)
    manager_page_url = urljoin(BASE_URL, manager_link)
    current_fund_name = scrape_fund_name(manager_page_url)
    print(f"    Fund Name: {current_fund_name}")

    # Determine URLs to scrape (quarter links or manager page if none)
    urls_to_scrape = quarter_links if quarter_links else [manager_page_url]
    if not quarter_links:
        logging.warning(f"No specific quarter links found for {manager_link}, attempting scrape on manager page directly: {manager_page_url}")

    total_urls_for_manager = len(urls_to_scrape)
    print(f"    Preparing to scrape data from {total_urls_for_manager} URL(s)...")

    # Scrape data for each URL associated with this manager
    for j, data_url in enumerate(urls_to_scrape):
        print(f"      -> Scraping URL [{j+1}/{total_urls_for_manager}]: {data_url}")
        # Call the main scraping function
        scrape_result = scrape_table_data_and_metadata(data_url)
        scraped_dfs_list = scrape_result.get('dataframes', [])
        filing_date = scrape_result.get('filing_date')
        quarter_str = scrape_result.get('quarter')

        if scraped_dfs_list:
            logging.info(f"Found {len(scraped_dfs_list)} table(s) on {data_url}.")
            print(f"        -- Found {len(scraped_dfs_list)} table(s) from this URL --")
            # Process each DataFrame found on this page
            for df_scraped in scraped_dfs_list:
                # Add metadata collected for this URL
                df_scraped['fund_name'] = current_fund_name
                df_scraped['filing_date'] = filing_date
                df_scraped['quarter'] = quarter_str
                # Append the processed DataFrame to the master list
                all_scraped_dataframes.append(df_scraped)
        else:
            logging.warning(f"No data tables were successfully extracted from {data_url}.")
            print(f"        -- No data tables extracted from this URL (check warnings/errors above) --")

        # Polite delay between *each data URL request* - IMPORTANT
        time.sleep(random.uniform(1.8, 4.8)) # Slightly increased delay

print("\n--- Finished Scraping All Filing Data ---")


# ==============================================================================
#               Execution Stage 4: Data Consolidation & Processing
# ==============================================================================
print("\n" + "="*70)
print(" Stage 4: Consolidating and Processing All Scraped Data")
print("="*70)

if not all_scraped_dataframes:
    logging.warning("No dataframes were scraped in Stage 3. Cannot proceed with processing.")
    final_processed_df = pd.DataFrame() # Ensure variable exists but is empty
else:
    # --- Combine all scraped dataframes ---
    logging.info(f"Combining {len(all_scraped_dataframes)} scraped dataframes...")
    print(f"Combining {len(all_scraped_dataframes)} scraped tables...")
    try:
        combined_df = pd.concat(all_scraped_dataframes, ignore_index=True, sort=False)
        logging.info(f"Initial combined DataFrame shape: {combined_df.shape}")
        print(f"Combined DataFrame shape: {combined_df.shape}.")
        # It's crucial to work on a copy for significant processing
        processed_df = combined_df.copy()
        del combined_df # Free up memory from the large intermediate list/df
    except Exception as e:
        logging.critical(f"CRITICAL Error during DataFrame concatenation: {e}", exc_info=True)
        processed_df = pd.DataFrame() # Assign empty df on critical error

# --- Process the combined dataframe (only if concatenation was successful) ---
if not processed_df.empty:
    logging.info("Starting data processing steps...")
    print("\nProcessing combined data...")

    # --- Standardize Column Names (apply cleaning to all columns) ---
    processed_df.columns = [_clean_header(col) for col in processed_df.columns]
    logging.info(f"Standardized column names: {processed_df.columns.tolist()}")

    # --- Filter for Common Stock ('COM') ---
    if 'cl' in processed_df.columns:
        original_rows = len(processed_df)
        # Ensure case-insensitivity and handle potential NaN
        processed_df = processed_df[processed_df['cl'].astype(str).str.upper() == 'COM']
        logging.info(f"Filtered for 'cl' == 'COM'. Rows reduced from {original_rows} to {len(processed_df)}.")
        print(f"Filtered for 'COM' stock. Rows remaining: {len(processed_df)}.")
    else:
        logging.warning("'cl' column not found, skipping filter for common stock.")

    # --- Data Type Conversion and Cleaning ---
    logging.info("Cleaning and converting data types...")
    # Shares (convert to numeric)
    if 'shares' in processed_df.columns:
        processed_df['shares'] = processed_df['shares'].astype(str).str.replace(',', '', regex=False)
        processed_df['shares_numeric'] = pd.to_numeric(processed_df['shares'], errors='coerce')
        nan_shares_count = processed_df['shares_numeric'].isna().sum()
        if nan_shares_count > 0:
            logging.warning(f"{nan_shares_count} 'shares' values could not be converted to numeric (set to NaN).")
    else:
        logging.warning("'shares' column not found. Creating 'shares_numeric' as NaN.")
        processed_df['shares_numeric'] = np.nan

    # Value Column (convert to numeric)
    value_col = 'value_usd_000'
    if value_col in processed_df.columns:
         processed_df[value_col] = processed_df[value_col].astype(str).str.replace(',', '', regex=False)
         processed_df[value_col] = pd.to_numeric(processed_df[value_col], errors='coerce')
         nan_value_count = processed_df[value_col].isna().sum()
         if nan_value_count > 0:
             logging.warning(f"{nan_value_count} '{value_col}' values could not be converted to numeric (set to NaN).")
    else:
         logging.warning(f"Value column '{value_col}' not found.")

    # Quarter Period (create sortable representation)
    if 'quarter' in processed_df.columns:
        def parse_quarter_to_period(q_str: Optional[str]) -> Optional[pd.Period]:
            """Parses 'QX YYYY ...' string to pandas Period object."""
            if not isinstance(q_str, str): return pd.NaT
            # More robust regex: Optional space, ignore case
            match = re.search(r'(Q[1-4])\s*(\d{4})', q_str, re.IGNORECASE)
            if match:
                q_num, year = match.group(1).upper(), match.group(2)
                try: return pd.Period(f"{year}{q_num}", freq='Q')
                except ValueError: return pd.NaT # Handle invalid year/quarter combo
            return pd.NaT # Return Not-a-Time if pattern not found
        processed_df['quarter_period'] = processed_df['quarter'].apply(parse_quarter_to_period)
        nan_quarter_parse_count = processed_df['quarter_period'].isna().sum()
        if nan_quarter_parse_count > 0:
             logging.warning(f"{nan_quarter_parse_count} 'quarter' values could not be parsed into sortable periods.")
    else:
        logging.warning("'quarter' column not found. Cannot sort chronologically.")
        processed_df['quarter_period'] = pd.NaT # Create column anyway for safety

    # --- Identify Stock Identifier ---
    # Determine the primary column for stock identification ('sym' preferred)
    if 'sym' in processed_df.columns:
        stock_id_col = 'sym'
    elif 'stock_symbol' in processed_df.columns:
        stock_id_col = 'stock_symbol'
    else:
        stock_id_col = None
    if stock_id_col:
        logging.info(f"Using '{stock_id_col}' as the primary stock identifier for grouping.")
    else:
        logging.error("CRITICAL: Neither 'sym' nor 'stock_symbol' column found. Cannot perform grouped calculations.")

    # --- Calculate Changes (Grouped Operations) ---
    # Proceed only if necessary columns are present and valid
    can_calculate_changes = (
        stock_id_col is not None and
        'shares_numeric' in processed_df.columns and
        'quarter_period' in processed_df.columns and
        processed_df['quarter_period'].notna().any() # Check if at least some quarters are parseable
    )

    if can_calculate_changes:
        logging.info(f"Calculating changes grouped by 'fund_name', '{stock_id_col}'...")
        print(f"Calculating quarterly changes grouped by fund and '{stock_id_col}'...")

        # IMPORTANT: Drop rows with NaNs in essential grouping/sorting keys to avoid errors
        essential_cols = ['fund_name', stock_id_col, 'quarter_period', 'shares_numeric']
        rows_before_drop = len(processed_df)
        processed_df.dropna(subset=[col for col in essential_cols if col in processed_df.columns], inplace=True)
        rows_after_drop = len(processed_df)
        if rows_before_drop > rows_after_drop:
            logging.warning(f"Dropped {rows_before_drop - rows_after_drop} rows with NaN in essential columns ({essential_cols}) before change calculation.")

        # Perform sorting on the cleaned data
        processed_df = processed_df.sort_values(
            by=['fund_name', stock_id_col, 'quarter_period'],
            ascending=True
        )

        # Define grouping columns
        group_cols = ['fund_name', stock_id_col]

        # Calculate 'change' (difference from previous quarter within group)
        if 'change' not in processed_df.columns: processed_df['change'] = pd.NA # Ensure column exists
        processed_df['change'] = processed_df.groupby(group_cols, observed=True)['shares_numeric'].diff().fillna(0)

        # Calculate 'pct_change'
        if 'pct_change' not in processed_df.columns: processed_df['pct_change'] = pd.NA # Ensure column exists
        # Calculate pct_change, handle potential division by zero
        pct_change_raw = processed_df.groupby(group_cols, observed=True)['shares_numeric'].pct_change()
        # Replace inf/-inf with NaN, then fill remaining NaN (first entry of group) with 0
        processed_df['pct_change'] = pct_change_raw.replace([np.inf, -np.inf], np.nan).fillna(0)

        # Infer 'inferred_transaction_type'
        if 'inferred_transaction_type' not in processed_df.columns: processed_df['inferred_transaction_type'] = pd.NA # Ensure column exists
        # Ensure 'change' is numeric before comparison
        change_numeric = pd.to_numeric(processed_df['change'], errors='coerce')
        conditions = [change_numeric < 0, change_numeric == 0, change_numeric > 0]
        choices = ['Sell', 'Hold', 'Buy']
        processed_df['inferred_transaction_type'] = np.select(conditions, choices, default='Unknown') # Default for NaN change

        logging.info("Calculations for 'change', 'pct_change', 'inferred_transaction_type' complete.")
        print("Quarterly change calculations complete.")

        # Optional: Convert 'change' to nullable integer type for cleaner output
        try: processed_df['change'] = processed_df['change'].astype(pd.Int64Dtype())
        except TypeError: pass # Keep as float if conversion fails

    else: # Cannot calculate changes
        logging.warning("Skipping quarterly change calculations due to missing essential columns or unparseable quarters.")
        print("Skipping quarterly change calculations (missing required columns or data).")
        # Ensure placeholder columns exist even if calculations are skipped
        for col in ['change', 'pct_change', 'inferred_transaction_type']:
             if col not in processed_df.columns: processed_df[col] = pd.NA


    # --- Final Column Formatting ---
    logging.info("Formatting final columns...")
    print("\nFormatting final columns for output...")
    # Rename stock identifier column to 'stock symbol' if needed
    if stock_id_col == 'sym' and 'sym' in processed_df.columns:
        processed_df.rename(columns={'sym': 'stock symbol'}, inplace=True)
        logging.info("Renamed 'sym' column to 'stock symbol'.")
    elif stock_id_col == 'stock_symbol' and 'stock symbol' not in processed_df.columns and 'sym' in processed_df.columns:
        logging.warning("Used 'stock_symbol' as ID, but column 'stock symbol' doesn't exist while 'sym' does. Check logic.")
    elif stock_id_col is None and 'sym' in processed_df.columns:
        # If we couldn't group but 'sym' exists, still rename for consistency if target includes 'stock symbol'
         if 'stock symbol' in TARGET_FINAL_COLUMNS:
              processed_df.rename(columns={'sym': 'stock symbol'}, inplace=True)
              logging.info("Renamed 'sym' to 'stock symbol' even though grouping failed.")


    # Ensure value column name is correct in target list if it was cleaned
    if 'value_usd_000' not in TARGET_FINAL_COLUMNS and 'value_usd_000' in processed_df.columns:
        # Adjust target list if needed, or assume 'value_usd_000' is intended
        logging.warning(f"Value column name used is 'value_usd_000', check consistency with TARGET_FINAL_COLUMNS: {TARGET_FINAL_COLUMNS}")


    # Select and order target columns, dropping others
    final_columns_existing = [col for col in TARGET_FINAL_COLUMNS if col in processed_df.columns]
    missing_target_cols = [col for col in TARGET_FINAL_COLUMNS if col not in final_columns_existing]
    if missing_target_cols:
        logging.warning(f"Final target columns missing from processed data: {missing_target_cols}")
        print(f"Warning: Final columns missing: {missing_target_cols}")

    final_processed_df = processed_df[final_columns_existing] # Select only existing target columns
    logging.info(f"Final selected columns: {final_processed_df.columns.tolist()}")
    print(f"Final DataFrame columns: {final_processed_df.columns.tolist()}")

    # Drop temporary columns explicitly if they weren't dropped by selection
    final_processed_df = final_processed_df.drop(columns=['shares_numeric', 'quarter_period'], errors='ignore')

else: # combined_df was empty
    logging.warning("Combined DataFrame was empty after scraping. No processing performed.")
    final_processed_df = pd.DataFrame() # Ensure variable exists but is empty


# ==============================================================================
#                        Execution Stage 5: Final Output
# ==============================================================================
print("\n" + "="*70)
print(" Stage 5: Final Output")
print("="*70)

if not final_processed_df.empty:
    # --- Print Head ---
    print(f"\n--- First 5 rows of the final DataFrame (Shape: {final_processed_df.shape}) ---")
    pd.set_option('display.max_columns', None) # Show all columns for head
    pd.set_option('display.width', 200)        # Adjust width for better viewing
    pd.set_option('display.max_colwidth', 70)  # Limit column width slightly for readability
    print(final_processed_df.head().to_string())
    pd.reset_option('display.max_colwidth') # Reset column width

    # --- Save to CSV ---
    print(f"\n--- Saving final DataFrame to '{CSV_FILENAME}' ---")
    try:
        final_processed_df.to_csv(CSV_FILENAME, index=False, encoding='utf-8-sig')
        print(f"Successfully saved data ({len(final_processed_df)} rows) to '{CSV_FILENAME}'")
    except Exception as e:
        print(f"CRITICAL ERROR: Failed to save the final DataFrame to CSV '{CSV_FILENAME}'. Error: {e}")

else:
    print("\nFinal DataFrame is empty or was not created due to errors. No CSV file saved.")

print("\n--- Script execution finished ---")


 Stage 1: Scraping Manager Links
Found 9 unique relative manager links.

 Stage 2: Scraping Quarter Links for Each Manager

[1/9] Getting quarter links for Manager: /manager/0001536411-duquesne-family-office-llc

[2/9] Getting quarter links for Manager: /manager/0001336528-pershing-square-capital-management-l-p

[3/9] Getting quarter links for Manager: /manager/0001350694-bridgewater-associates-lp



[4/9] Getting quarter links for Manager: /manager/0001067983/cusip/037833100

[5/9] Getting quarter links for Manager: /manager/0001067983-berkshire-hathaway-inc

[6/9] Getting quarter links for Manager: /manager/0001364742-blackrock-inc

[7/9] Getting quarter links for Manager: /manager/0001167483-tiger-global-management-llc

[8/9] Getting quarter links for Manager: /manager/0001656456-appaloosa-lp

[9/9] Getting quarter links for Manager: /manager/0001649339-scion-asset-management-llc

Finished getting quarter links for 9 managers processed.

 Stage 3: Scraping Table Data and Metadata for Each Filing

>>> Processing Manager 1/9: /manager/0001536411-duquesne-family-office-llc <<<
    Fund Name: Duquesne Family Office LLC
    Preparing to scrape data from 47 URL(s)...
      -> Scraping URL [1/47]: https://13f.info/13f/000153641125000006-duquesne-family-office-llc-q4-2024
        -- Found 1 table(s) from this URL --
      -> Scraping URL [2/47]: https://13f.info/13f/000153641124000009-


>>> Processing Manager 4/9: /manager/0001067983/cusip/037833100 <<<
    Fund Name: Berkshire Hathaway Inc
    Preparing to scrape data from 1 URL(s)...
      -> Scraping URL [1/1]: https://13f.info/manager/0001067983/cusip/037833100
        -- No data tables extracted from this URL (check warnings/errors above) --

>>> Processing Manager 5/9: /manager/0001067983-berkshire-hathaway-inc <<<
    Fund Name: Berkshire Hathaway Inc
    Preparing to scrape data from 50 URL(s)...
      -> Scraping URL [1/50]: https://13f.info/13f/000095012325002701-berkshire-hathaway-inc-q4-2024
        -- Found 1 table(s) from this URL --
      -> Scraping URL [2/50]: https://13f.info/13f/000095012324011775-berkshire-hathaway-inc-q3-2024
        -- Found 1 table(s) from this URL --
      -> Scraping URL [3/50]: https://13f.info/13f/000095012324008740-berkshire-hathaway-inc-q2-2024
        -- Found 1 table(s) from this URL --
      -> Scraping URL [4/50]: https://13f.info/13f/000095012324005622-berkshire-hath

<ipython-input-1-79a371a4c81f>:587: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_df = pd.concat(all_scraped_dataframes, ignore_index=True, sort=False)


Combined DataFrame shape: (233917, 13).

Processing combined data...
Filtered for 'COM' stock. Rows remaining: 141438.


<ipython-input-1-79a371a4c81f>:620: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  processed_df['shares'] = processed_df['shares'].astype(str).str.replace(',', '', regex=False)


Calculating quarterly changes grouped by fund and 'sym'...
Quarterly change calculations complete.

Formatting final columns for output...
Final DataFrame columns: ['fund_name', 'filing_date', 'quarter', 'stock symbol', 'cl', 'value_usd_000', 'shares', 'change', 'pct_change', 'inferred_transaction_type']

 Stage 5: Final Output

--- First 5 rows of the final DataFrame (Shape: (138401, 10)) ---
           fund_name filing_date               quarter stock symbol   cl  value_usd_000     shares   change  pct_change inferred_transaction_type
233385  APPALOOSA LP  11/14/2016  Q3 2016 13F Holdings         AABA  COM          77580  1800000.0        0    0.000000                      Hold
233343  APPALOOSA LP   2/14/2017  Q4 2016 13F Holdings         AABA  COM          52205  1350000.0  -450000   -0.250000                      Sell
233281  APPALOOSA LP   5/12/2017  Q1 2017 13F Holdings         AABA  COM         102102  2200000.0   850000    0.629630                       Buy
233209  APPALOOSA L